<a href="https://colab.research.google.com/github/Ismael-Haque/selenium_webscraping/blob/main/selenium_scrapeme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
w

In [ ]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set the target URL
# url = "https://www.pcmag.com/news"
url = "https://scrapeme.live/shop/"

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
driver.get(url)

In [ ]:
li_elements = driver.find_elements(By.XPATH, '//*[@id="main"]/div[1]/nav/ul/li')
for li_element in li_elements:
  print(li_element.text)

1
2
3
4
…
46
47
48
→


In [ ]:
li_elements[-1].text

'→'

In [ ]:
next_button_exists = True
link_of_products = []

while next_button_exists:
  try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div[1]/nav/ul/li')))
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/ul/li/a')))

    a_elements = driver.find_elements(By.XPATH, '//*[@id="main"]/ul/li/a[1]')
    for a_element in a_elements:
      link = a_element.get_attribute("href")
      link_of_products.append(link)
      print(link)
    print(" page ends "*100)

    # get paginations
    nav_list_els = driver.find_elements(By.XPATH, '//*[@id="main"]/div[1]/nav/ul/li')
    nav_list_els[-1].click()

    # check if it's the last page
    if int(nav_list_els[-1].text) == 48:
      next_button_exists = False
  except:
    pass

In [ ]:
print(link_of_products)

['https://scrapeme.live/shop/Bulbasaur/', 'https://scrapeme.live/shop/Ivysaur/', 'https://scrapeme.live/shop/Venusaur/', 'https://scrapeme.live/shop/Charmander/', 'https://scrapeme.live/shop/Charmeleon/', 'https://scrapeme.live/shop/Charizard/', 'https://scrapeme.live/shop/Squirtle/', 'https://scrapeme.live/shop/Wartortle/', 'https://scrapeme.live/shop/Blastoise/', 'https://scrapeme.live/shop/Caterpie/', 'https://scrapeme.live/shop/Metapod/', 'https://scrapeme.live/shop/Butterfree/', 'https://scrapeme.live/shop/Weedle/', 'https://scrapeme.live/shop/Kakuna/', 'https://scrapeme.live/shop/Beedrill/', 'https://scrapeme.live/shop/Pidgey/', 'https://scrapeme.live/shop/Pidgeotto/', 'https://scrapeme.live/shop/Pidgeot/', 'https://scrapeme.live/shop/Rattata/', 'https://scrapeme.live/shop/Raticate/', 'https://scrapeme.live/shop/Spearow/', 'https://scrapeme.live/shop/Fearow/', 'https://scrapeme.live/shop/Ekans/', 'https://scrapeme.live/shop/Arbok/', 'https://scrapeme.live/shop/Pikachu/', 'https:/

In [ ]:
import numpy as np

In [ ]:
product_details = []

for product_link in link_of_products:
  driver.get(product_link)
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.summary.entry-summary')))

  product = driver.find_element(By.CSS_SELECTOR, 'div.summary.entry-summary')
  name = None
  description = None
  categories = None
  price = None
  tags = None

  try:
    name = product.find_element(By.CSS_SELECTOR, "h1.product_title.entry-title").text
  except:
    pass

  try:
    description = product.find_element(By.CSS_SELECTOR, "div.woocommerce-product-details__short-description > p").text
  except:
    pass

  try:
    categories = product.find_element(By.CLASS_NAME, 'posted_in').text
  except:
    pass

  try:
    price = product.find_element(By.CSS_SELECTOR, 'span.woocommerce-Price-amount.amount').text
  except:
    pass

  try:
    tags = product.find_element(By.CLASS_NAME, "tagged_as").text
  except:
    pass

  product_details.append(
      {
          "name": name,
          "price" : price,
          "description" : description,
          "categories" : categories,
          "tags" : tags
      }
  )

  print(
      {
          "name": name,
          "price" : price,
          "description" : description,
          "categories" : categories,
          "tags" : tags
      }
  )
  print("--"*100)

In [ ]:
import pandas as pd

df = pd.DataFrame(product_details)
df

,name,price,description,categories,tags
0,Bulbasaur,£63.00,Bulbasaur can be seen napping in bright sunlig...,"Categories: Pokemon, Seed","Tags: bulbasaur, Overgrow, Seed"
1,Ivysaur,£87.00,There is a bud on this Pokémon’s back. To supp...,"Categories: Pokemon, Seed","Tags: ivysaur, Overgrow, Seed"
2,Venusaur,£105.00,There is a large flower on Venusaur’s back. Th...,"Categories: Pokemon, Seed","Tags: Overgrow, Seed, venusaur"
3,Charmander,£48.00,The flame that burns at the tip of its tail is...,"Categories: Lizard, Pokemon","Tags: Blaze, charmander, Lizard"
4,Charmeleon,£165.00,Charmeleon mercilessly destroys its foes using...,"Categories: Flame, Pokemon","Tags: Blaze, charmeleon, Flame"
...,...,...,...,...,...
750,Marshadow,£107.00,It sinks into the shadows of people and Pokémo...,"Categories: Gloomdweller, Pokemon","Tags: Gloomdweller, marshadow, Technician"
751,Poipole,£130.00,An Ultra Beast that lives in a different world...,"Categories: Poison Pin, Pokemon","Tags: Beast Boost, poipole, Poison Pin"
752,Naganadel,£30.00,"One kind of Ultra Beast, it fires a glowing, v...","Categories: Poison Pin, Pokemon","Tags: Beast Boost, naganadel, Poison Pin"
753,Stakataka,£190.00,"When stone walls started moving and attacking,...","Categories: Pokemon, Rampart","Tags: Beast Boost, Rampart, stakataka"


In [ ]:
df.to_csv("scrapme_pokemon.csv")